<a href="https://colab.research.google.com/github/boleamol/Twitter_Sentiment_Analysis/blob/main/Twitter_Sentiment_Analysis_Using_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pwd

/content


In [2]:
#changing the working directory and entering to dataset directory for my google drive
%cd drive/MyDrive/kaggle/twitter_sentiment_analysis/
!pwd

/content/drive/MyDrive/kaggle/twitter_sentiment_analysis
/content/drive/MyDrive/kaggle/twitter_sentiment_analysis


In [3]:
!ls

test.csv  train.csv


In [4]:
!pip install tensorflow===2.1.0

     |████████████████████████████████| 421.8 MB 25 kB/s 
     |████████████████████████████████| 448 kB 45.3 MB/s 
     |████████████████████████████████| 3.8 MB 42.5 MB/s 
     |████████████████████████████████| 50 kB 5.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=1c9021f9ff2cbedb3256f7c904ea470714d8cad9c16affb33b6327c9f87d0524
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [5]:
import tensorflow as tf

In [6]:
tf.__version__

'2.1.0'

In [7]:
import pandas as pd

In [8]:
df=pd.read_csv('train.csv')

In [9]:
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [10]:
###Drop Nan Values
df=df.dropna()

In [11]:
## Getting Independent Features in X
X=df.drop('label',axis=1)

In [12]:
## Get the Dependent features
y=df['label']

In [13]:
X.shape

(31962, 2)

In [14]:
y.shape

(31962,)

In [15]:
y.value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [17]:
### Vocabulary size
voc_size=5000

In [18]:
messages=X.copy()

In [20]:
messages['tweet'][1]

"@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked"

In [21]:
messages.reset_index(inplace=True)

In [22]:
import nltk
import re
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['tweet'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [26]:
corpus

['user father dysfunct selfish drag kid dysfunct run',
 'user user thank lyft credit use caus offer wheelchair van pdx disapoint getthank',
 'bihday majesti',
 'model love u take u time ur',
 'factsguid societi motiv',
 'huge fan fare big talk leav chao pay disput get allshowandnogo',
 'user camp tomorrow user user user user user user user danni',
 'next school year year exam think school exam hate imagin actorslif revolutionschool girl',
 'love land allin cav champion cleveland clevelandcavali',
 'user user welcom gr',
 'ireland consum price index mom climb previou may blog silver gold forex',
 'selfish orlando standwithorlando pulseshoot orlandoshoot biggerproblem selfish heabreak valu love',
 'get see daddi today day gettingf',
 'user cnn call michigan middl school build wall chant tcot',
 'comment australia opkillingbay seashepherd helpcovedolphin thecov helpcovedolphin',
 'ouch junior angri got junior yugyoem omg',
 'thank paner thank posit',
 'retweet agre',
 'friday smile around

In [27]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4001, 4022, 3026, 2959, 4699, 2383, 3026, 1335],
 [4001,
  4001,
  2852,
  4660,
  4914,
  1041,
  3055,
  4815,
  2604,
  3405,
  2817,
  1867,
  3492],
 [4172, 463],
 [3704, 1545, 1161, 4447, 1161, 3218, 241],
 [1517, 1603, 1155],
 [4844, 4724, 3017, 398, 1138, 2666, 2087, 1324, 3990, 2151, 1583],
 [4001, 461, 4661, 4001, 4001, 4001, 4001, 4001, 4001, 4001, 4259],
 [1942, 790, 1196, 1196, 3089, 4804, 790, 3089, 3658, 4294, 428, 4025, 224],
 [1545, 3233, 3018, 4504, 1876, 3466, 1352],
 [4001, 4001, 980, 4771],
 [2113, 803, 4604, 4662, 200, 4441, 2097, 1389, 553, 155, 679, 4270],
 [2959, 2941, 1835, 1818, 3921, 166, 2959, 3335, 2155, 1545],
 [2151, 1539, 4278, 2589, 3242, 1660],
 [4001, 3550, 4881, 498, 4237, 790, 1714, 1662, 1572, 3977],
 [1626, 1459, 956, 3148, 3077, 4684, 3077],
 [142, 221, 2033, 2862, 221, 1052, 3785],
 [2852, 4282, 2852, 4680],
 [824, 13],
 [2194, 1208, 1156, 1196, 2374, 4001, 4001, 1119, 1621, 2879],
 [687, 2796, 505, 130, 3449],
 [2242,
  2879,
  1270,
  4756,

In [28]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2383 3026 1335]
 [   0    0    0 ... 2817 1867 3492]
 [   0    0    0 ...    0 4172  463]
 ...
 [   0    0    0 ... 3297 4804 1055]
 [   0    0    0 ...  829 3987 2215]
 [   0    0    0 ... 2852 4001 4036]]


In [29]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0, 4001, 4022, 3026, 2959, 4699, 2383, 3026, 1335], dtype=int32)

In [30]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
len(embedded_docs),y.shape

(31962, (31962,))

In [32]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [33]:
X_final.shape,y_final.shape

((31962, 20), (31962,))

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [35]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Train on 21414 samples, validate on 10548 samples
Epoch 1/15
21414/21414 [==============================] - 26s 1ms/sample - loss: 0.1953 - accuracy: 0.9393 - val_loss: 0.1486 - val_accuracy: 0.9507
Epoch 2/15
21414/21414 [==============================] - 23s 1ms/sample - loss: 0.0985 - accuracy: 0.9645 - val_loss: 0.1438 - val_accuracy: 0.9526
Epoch 3/15
21414/21414 [==============================] - 23s 1ms/sample - loss: 0.0778 - accuracy: 0.9735 - val_loss: 0.1576 - val_accuracy: 0.9484
Epoch 4/15
21414/21414 [==============================] - 23s 1ms/sample - loss: 0.0598 - accuracy: 0.9796 - val_loss: 0.1853 - val_accuracy: 0.9479
Epoch 5/15
21414/21414 [==============================] - 23s 1ms/sample - loss: 0.0472 - accuracy: 0.9850 - val_loss: 0.2129 - val_accuracy: 0.9501
Epoch 6/15
21414/21414 [==============================] - 23s 1ms/sample - loss: 0.0391 - accuracy: 0.9885 - val_loss: 0.2200 - val_accuracy: 0.9425
Epoch 7/15
21414/21414 [==============================] 

In [36]:
y_pred=model1.predict_classes(X_test)

In [37]:
from sklearn.metrics import confusion_matrix

In [38]:
confusion_matrix(y_test,y_pred)

array([[9613,  193],
       [ 381,  361]])

In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9455821008722033